## Классификация имён через n-граммы и LSTM.
### Выполнил: Хорин М. А.<br>

In [1]:
from string import ascii_lowercase, ascii_uppercase
from random import shuffle 
import numpy as np
from nltk.util import ngrams
from nltk.classify import NaiveBayesClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from sklearn.metrics import f1_score, accuracy_score

Using TensorFlow backend.


<b>1. </b>Произведём предварительную обработку данных. Сначала удалим те имена, которые являются и мужскими, и женскими одновременно. 

In [2]:
# считываем данные
male_names = open('male.txt').read().split('\n')
female_names = open('female.txt').read().split('\n')

male_names = [mname for mname in male_names] # приводим ты имена к нижнему регистру
female_names = [fname for fname in female_names]

print('Всего мужских имён:', len(male_names))
print('Всего женских имён:', len(female_names))

def clear_ambiguous_names(male_names, female_names):
    """
    метод производит очистку списков имён от неоднозначных имён
    """
    for mname in male_names: # идём по списку мужских имён
        if mname in female_names: # удаляем имя, если оно также содержится в списке женских имен
            female_names.remove(mname)
            male_names.remove(mname)
            mname=mname.lower()
    for fname in female_names: # идём по списку женских имён
        if fname in male_names: # удаляем имя, если оно также содержнится в списке мужских имён
            male_names.remove(fname)
            female_names.remove(fname) 
    return male_names, female_names # возвращаем очищенные списки

Всего мужских имён: 2943
Всего женских имён: 5001


In [3]:
male_names, female_names = clear_ambiguous_names(male_names, female_names)
print('Всего мужских имён:', len(male_names))
print('Всего женских имён:', len(female_names))

Всего мужских имён: 2578
Всего женских имён: 4636


Как можно заметить, среди всех имён встретились 365 неоднозначных, поэтому они были удалены.<br><br>Сделаем метку с полом к каждому имени, так как это понадобится для обучения модели. Также сделаем единый список имён, отсортированный по алфавиту, чтобы сформировать тестовое множество имён.

In [4]:
labeled_males = [(mname, 'male') for mname in male_names] # добавляем метки к мужским именам
labeled_females = [(fname,'female') for fname in female_names] # добавляем метки к женским именам

labeled_names = labeled_males + labeled_females # объединяем списки
labeled_names.sort() # сортируем
labeled_names[:10]

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abagael', 'female'),
 ('Abagail', 'female'),
 ('Abbe', 'female'),
 ('Abbi', 'female'),
 ('Abbot', 'male'),
 ('Abbott', 'male'),
 ('Abdel', 'male'),
 ('Abdul', 'male')]

Приступим к созданию тестового множества, в котором будет находиться 20% от имён на каждую букву алфавита. Сначала сделаем список список имён, начинающихся на каждую букву алфавита, чтобы упростить основную задачу.

In [5]:
sets = [] # инициируем основной список
for letter in ascii_uppercase: # идём по алфавиту
    letter_list = [] # создаем список, в который будут помещаться имена, начинающиеся с рассматриваемой буквы
    for name in labeled_names: # идём по размеченному списку имён
        if name[0].startswith(letter): # добавляем имя в список, если оно начинается с рассматриваемой буквы
            letter_list.append(name) 
    sets.append(letter_list) # добавляем список имён в основной список
    
sets[0][:10] # список имён, начинающихся на A

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abagael', 'female'),
 ('Abagail', 'female'),
 ('Abbe', 'female'),
 ('Abbi', 'female'),
 ('Abbot', 'male'),
 ('Abbott', 'male'),
 ('Abdel', 'male'),
 ('Abdul', 'male')]

Теперь создадим тестовое множество, содержащее 20% от общего количества имён на каждую букву. 

In [6]:
test_data = [] # список данных для тестирования модели
for letter_set in sets: # идём по спискам имён, начинающихся с определенной буквы
    shuffle(letter_set) # перемешиваем имена в них, чтобы исключить попадание имён, относящихся только к одному полу
    test_data += letter_set[:round(len(letter_set)*0.2)] # добавляем 20% имён в список
    
test_set = set(test_data) # создаеём тестовое множество

In [7]:
train_data = labeled_names # убираем элементы тестовых данных из исходных данных и получаем тренировочные данные
for element in test_data:
    train_data.remove(element)

In [8]:
print('Количество тренировочных данных:', len(train_data))
print('Количество данных для тестирования:', len(test_data))

Количество тренировочных данных: 5774
Количество данных для тестирования: 1440


<b>2. </b>Приступим к классификации имён с помощью наивного байесовского классификатора.

В качестве признаков необходимо использовать символьные н-граммы от имён, а сравнивать результаты классификации необходимо по F-мере и аккуратности. Создадим две функции: одна будет создавать признаки, используя символьные н-граммы, а вторая будет проводить обучение классификатора и расчёт метрик качества классификации.

In [9]:
def make_features(name, n):
    """
    Функция создаёт признаки для классификатора, раскладывая имя на н-граммы 
    """
    return dict([(''.join(ngram), True) for ngram in list(ngrams(name, n))])

def compute_metrics(test_data, train_data, n):
    """
    Функция осуществляет формирование множества признаков для обучения классификатора, обучает классификатор, а также рассчитывает 
    метрики качества классификации в зависимости от того, какой из полов мы считаем положительным классом (+1), а какой отрицательным
    (-1) 
    """
    train_featureset = [(make_features(name, n), gender) for (name, gender) in train_data] # создание тренировочного множества признаков
    classifier = NaiveBayesClassifier.train(train_featureset) # обучение классификатора
    
    print('Classification metrics:')
    genders = ['female', 'male'] # список полов (классов)   
    for gend in genders: # выбираем по-очереди каждый из классов в качестве положительного и рассчитываем в зависимости от этого метрики качества классификации
    
    # ячейки таблицы сопряженности бинарного классификатора
        tp = 0 # True positive classifications
        tn = 0 # True negative classifications
        fp = 0 # False positive classifications
        fn = 0 # False negative classifications
        
        if gend=='female': # считаем метрики, считая, что женский пол является положительным классом
            for (name, gender) in test_data: # идём по тестовым данным
                guess = classifier.classify(make_features(name, n)) # делаем предсказание и изменяем значение в ячейке таблицы сопряженности      
                if (guess=='female') & (gender=='female'):
                    tp += 1
                elif (guess=='female') & (gender=='male'):
                    fp += 1
                elif (guess=='male') & (gender=='female'):
                    fn += 1
                elif (guess=='male') & (gender=='male'):
                    tn += 1
            
            # рассчитываем метрики    
            precision_1 = tp/(tp+fp) # точность
            recall_1 = tp/(tp+fn) # полноту
            f_measure_1 = (2*precision_1*recall_1)/(precision_1+recall_1) # f-меру
            print('\tF-measure for %s classification - %f' % (gend, f_measure_1))
                
        elif gend=='male': # повторяем такие-же действия, считая мужской пол позитивным классом
            for (name, gender) in test_data:
                guess = classifier.classify(make_features(name, n))
                if (guess=='male') & (gender=='male'):
                    tp += 1
                elif (guess=='male') & (gender=='female'):
                    fp += 1
                elif (guess=='female') & (gender=='male'):
                    fn += 1
                elif (guess=='female') & (gender=='female'):
                    tn += 1
            cls_accuracy = (tp+tn)/(tp+tn+fp+fn) # расчёт аккуратности
            precision_2 = tp/(tp+fp)
            recall_2 = tp/(tp+fn)
            f_measure_2 = (2*precision_2*recall_2)/(precision_2+recall_2)
            
            # выводим значения метрик качества классификации
            print('\tF-measure for %s classification - %f' % (gend, f_measure_2))
            print('\tClassification accuracy - %f' % (cls_accuracy))
    return classifier # возвращаем классификатор

2-GRAMS

In [10]:
clf_2gram = compute_metrics(test_data, train_data, 2)

Classification metrics:
	F-measure for female classification - 0.837616
	F-measure for male classification - 0.661670
	Classification accuracy - 0.780556


3-GRAMS

In [11]:
clf_3gram = compute_metrics(test_data, train_data, 3)

Classification metrics:
	F-measure for female classification - 0.873107
	F-measure for male classification - 0.748187
	Classification accuracy - 0.831250


4-GRAMS

In [12]:
clf_4gram = compute_metrics(test_data, train_data, 4)

Classification metrics:
	F-measure for female classification - 0.864594
	F-measure for male classification - 0.695260
	Classification accuracy - 0.812500


Как можно заметить выше, наибольшая аккуратность и F-мера при классификации достигаются, если в качестве признаков использовать символьные 3-граммы, составленные от имён. Предположительно, это связано с тем, что именно 3-граммы наиболее полно и однозначно (по сравнению с 2 и 4-граммами) позволяют идентифицировать пол по имени, так как в отличие от использования 2-грам, многие из которых как у мужских, так и у женских имён могут совпадать, а также в отличие от 4-грамм, которых у имён может быть либо немного, а то и вовсе отсутствовать (не все имена очень длинные), составление 3-грамм получается оптимальным по их количеству и информативности относительно пола. <br><br>
Тем не менее, классификатор будет совершать ошибки, если имя не позволяет создать ни одной символьной n-граммы, а также, если отличие в мужском и женском имени незначительно (небольшая разница в написании или наличие опечаток), или же, если имя может относиться к обоим полам. Также классификация будет неверной, если пытаться классифицировать имена, написанные на другом алфавите.

<b>3. </b> Построим сеть с двумя слоями LSTM для определения пола. Произведём обработку имён: каждое имя представим в виде бинарного вектора Х количество букв в алфавите х максимальная длина имени. Если первая буква имени а, то Х[1][1]=1, если вторая - b, то X[2][1]=1 и т.д. Вышеописанная обработка имён реализуется функцией preprocess_data, которая также производит обработку меток полов посредством one-hot кодирования.

In [13]:
def preprocess_data(data, dim1, dim2):
    """
    функция производит обработку имён и меток пола для них
    """
    namesToTransform = [] # список для хранения обработанных имён
    labelesToTransform = [] # список для хранения обработанных меток полов
    
    char_indices = dict((ch, i) for i, ch in enumerate(ascii_lowercase)) # формируем словарь буква алфавита : её индекс
    
    for (name, gender) in data: # по каждому имени с меткой
        x = np.zeros((dim1, dim2), dtype=int) # формируем исходный бинарный вектор
        y = np.zeros((2, ), dtype=int) # формируем вектор для перекадированной метки пола
        
        for c in name: # по каждому символу в имени
            if c not in ['-',' ', "'"]: # изменяем значения элементов в бинарном векторе в соответствии с идексами символов
                x[char_indices[c.lower()]][0]=1
        
        namesToTransform.append(x) # добавляем бинарный вектор в список
        
        # кодируем метку пола: если пол женский [1 0], [0 1] - иначе
        if gender=='female':
            y[0]=1
        else:
            y[1]=1
        labelesToTransform.append(y)  # добавляем закодированную метку в список  
    
    # приводим списки к типу ndarray
    preprocessed_names = np.array(namesToTransform, dtype=int) 
    preprocessed_labeles = np.array(labelesToTransform, dtype=int)
    
    return preprocessed_names, preprocessed_labeles

In [14]:
alphabet_len = len(ascii_lowercase) # длина алфавита
max_len = len(max(female_names+male_names, key=len)) # максимальная длина имени

# формируем тренировочные и тестовые наборы
X_train, y_train = preprocess_data(train_data, alphabet_len, max_len)
X_test, y_test = preprocess_data(test_data, alphabet_len, max_len)

После формирования наборов сравним результаты классификаций, осуществляемых нейронными сетями с разным числом узлов на слоях и разных значениях дропаута по F-мере и аккуратности. Реализованная функция check_model формирует нейронную сеть с двумя слоями lstm, обучает её, делает предсказания и измеряет качество классификации при заданном количестве узлов на слоях сети и значении дропаута.

In [15]:
def check_model(neurons, dropout, X_train, y_train, X_test, y_test):
    print('Checking model with %d neurons and %f dropout rate...' % (neurons, dropout))
    
    # создаём нейронную сеть
    print('Building the model...') 
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(alphabet_len, max_len), return_sequences=True)) # первый lstm слой
    model.add(Dropout(dropout))
    model.add(LSTM(neurons, return_sequences=False)) # второй lstm слой
    model.add(Dropout(dropout))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy', 'fmeasure'])
    print('\tModel built...')
    
    # обучаем модель
    print('Training the model...')
    model.fit(X_train, y_train, batch_size=32, nb_epoch=8, verbose=0)
    print('\tModel trained...')
    
    # делаем предсказания
    print('Making predictions...')
    predictions = model.predict(X_test, verbose=0)
    pred_labels = []
    for element in predictions: # метод predict возвращает вероятности. сделаем список меток пола 1 - женский 0 - мужской
        if element[0] > element[1]:
            pred_labels.append(1)
        else:
            pred_labels.append(0)
    print('\tPredictions made...')
    
    test_labels = [elem[0] for elem in y_test] # метки пола для тестовых данных
    print('Evaluating classification results...') 
    print('\tAccuracy:', accuracy_score(test_labels, pred_labels)) # измерение аккуратности
    print('\tF-score:', f1_score(test_labels, pred_labels)) # измерение f-меры

Оценим качество классификации при 100 узлах на слоях сети и значении дропаута 0.1.

In [16]:
%%time
check_model(100, 0.1, X_train, y_train, X_test, y_test)

Checking model with 100 neurons and 0.100000 dropout rate...
Building the model...
	Model built...
Training the model...
	Model trained...
Making predictions...
	Predictions made...
Evaluating classification results...
	Accuracy: 0.659027777778
	F-score: 0.766967252017
Wall time: 2min 34s


Оценим качество классификации при 400 узлах на слоях сети и значении дропаута 0.9.

In [17]:
%%time
check_model(400, 0.9, X_train, y_train, X_test, y_test)

Checking model with 400 neurons and 0.900000 dropout rate...
Building the model...
	Model built...
Training the model...
	Model trained...
Making predictions...
	Predictions made...
Evaluating classification results...
	Accuracy: 0.646527777778
	F-score: 0.785322648671
Wall time: 16min 31s


Как можно заметить выше, при большом значении дропаута аккуратность классификации становится меньше, независимо от того, много ли узлов на слоях нейронной сети.

Данная нейронная сеть часто ошибается в классификации мужских имён. Вероятно, это связано с тем, что в тренировочном наборе данных преобладают женские имена.

<b>4. </b>Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравнивая результаты классификации, можно заметить, что наивный байесовский классификатор в данном случае лучше справляется с задачей классификации пола по имени, чем нейронная сеть. <br><br>Вероятно, наивный байесовский классификатор лучше, так как предположение о независимости н-грамм в имени логично: ведь, вероятно, верно, что зависимость между н-граммами имени довольно мала.
Также данный алгоритм не требует большого количества данных для обучения, он прост в понимании и работе (проще в качестве признаков использовать предположительно независимые между собой н-граммы, чем представлять каждое имя в виде матрицы). Нейронная сеть же требует большое количество правильно предобработанных данных. Более того, очень важна архитектура сети и подбор гиперпараметров. Считаю, что можно улучшить качество работы сети, если более тщательно обработать данные перед обучением сети и настроить её параметры.<br><br>
Также могу предположить другой вариант низкого качества работы нейронной сети: кривая установка библиотек tenserflow и keras на windows 10.